In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

DATAPATH = "../input/megaGymDataset.csv"

In [12]:
dataset = pd.read_csv(DATAPATH)
dataset = dataset.fillna('')
dataset.head()

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,


In [13]:
# Function to recommend top 10 exercises based on similarity scores
def recommend_exercises(user_input):
    features = dataset['Type'] + ' ' + dataset['BodyPart'] + ' ' + dataset['Equipment'] + ' ' + dataset['Level']
    
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(features)

    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    indices = dataset.index[
        (dataset['Type'] == user_input[0]) &
        (dataset['BodyPart'] == user_input[1]) &
        (dataset['Equipment'].str.contains(user_input[2], case=False)) &
        (dataset['Level'] == user_input[3])
    ].tolist()

    exercise_scores = list(enumerate(cosine_sim[indices[0]]))
    exercise_scores = sorted(exercise_scores, key=lambda x: x[1], reverse=True)
    exercise_scores = exercise_scores[:10]  # Exclude the exercise itself and select top 10
    exercise_indices = [i[0] for i in exercise_scores]
    
    return dataset.loc[exercise_indices]

In [14]:
user_input = ["Strength", "Chest", "Cable", "Intermediate"]
recommend_exercises(user_input)

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating
1073,1073,Low-cable cross-over,The low-cable cross-over is an isolation movem...,Strength,Chest,Cable,Intermediate,9.1
1074,1074,Incline cable chest fly,The incline cable chest fly is an isolation mo...,Strength,Chest,Cable,Intermediate,9.0
1075,1075,Cable Crossover,The cable cross-over is an isolation movement ...,Strength,Chest,Cable,Intermediate,8.8
1078,1078,Single-arm cable cross-over,The single-arm cable cross-over is an isolatio...,Strength,Chest,Cable,Intermediate,8.6
1081,1081,Lying cable chest fly,The lying cable chest fly is an isolation move...,Strength,Chest,Cable,Intermediate,7.3
1082,1082,Paul Carter Cable Cross-Over,The cable cross-over is an isolation movement ...,Strength,Chest,Cable,Intermediate,
1083,1083,One-Arm Cable Crossover - Gethin Variation,The single-arm cable cross-over is an isolatio...,Strength,Chest,Cable,Intermediate,
1084,1084,Incline Cable Fly - Gethin Variation,The incline cable chest fly is an isolation mo...,Strength,Chest,Cable,Intermediate,
1085,1085,Cable Cross-Over - Gethin Variation,The cable cross-over is an isolation movement ...,Strength,Chest,Cable,Intermediate,
1086,1086,Seated cable chest fly,The seated cable chest fly is an isolation mov...,Strength,Chest,Cable,Intermediate,
